In [6]:
import numpy as np
import csv
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd

In [7]:
import tensorflow as tf
print(tf.__version__)
news = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

2.3.1


In [9]:
news_corpus = pd.read_csv("train.csv", index_col=0)
news_corpus = news_corpus.loc[:,['text', 'label']]
news_corpus.head()

,text,label
id,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
news_corpus.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [14]:
news = news_corpus.text.to_numpy(dtype = str)
labels = news_corpus.label.to_numpy()

In [19]:
news_cleaned = []
for text in np.nditer(news):
    for word in stopwords:
        removable = " " + word + " "
        text = str(text).replace(removable, " ")
    news_cleaned.append(text)

In [22]:
news = np.array(news_cleaned, dtype = str)

In [32]:
##Variables
VOCAB_SIZE = 150000
EMBEDDING_DIM = 100
MAX_LENGTH = 2000
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOK = "<OOV>"

In [26]:
from sklearn.model_selection import train_test_split
news_train, news_test, labels_train, labels_test = train_test_split(news, labels, test_size = 0.2, random_state = 101)

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token = OOV_TOK)
tokenizer.fit_on_texts(news_train)
word_index = tokenizer.word_index

In [42]:
news_sequences = tokenizer.texts_to_sequences(news_train)

In [43]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_news_sequences = pad_sequences(news_sequences, maxlen = MAX_LENGTH, padding = PADDING_TYPE, truncating = TRUNC_TYPE)

In [44]:
test_sequences = tokenizer.texts_to_sequences(news_test)
test_padded = pad_sequences(test_sequences, maxlen = MAX_LENGTH, padding = PADDING_TYPE, truncating = TRUNC_TYPE)

In [52]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length = MAX_LENGTH))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv1D(500, 10, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size = 10))
model.add(tf.keras.layers.LSTM(64, return_sequences = True))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(24, activation = 'relu'))
model.add(tf.keras.layers.Dense(6, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))


In [53]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 100)         15000000  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000, 100)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1991, 500)         500500    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 199, 500)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 199, 64)           144640    
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_5 (Dense)              (None, 24)               

In [55]:
NUM_EPOCHS = 10
history = model.fit(padded_news_sequences, labels_train, epochs = NUM_EPOCHS, validation_data = (test_padded, labels_test), verbose = 2)

Epoch 1/10


KeyboardInterrupt: 